# To calculate parameters for afni bandpass function and write them in ortho file.txt


In [1]:
# rp_at0009_epi_s04_d0001.txt
# todo recuperer le seul fichier texte de results functionnal
# rp_at0009_epi_s+04_d0001.txt

mocoFile = '/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal/rp_at0009_epi_s04_d0001.txt'

In [2]:
import pandas as pd
import numpy as np

mocodf = pd.read_csv(mocoFile, header=None, sep='  ',engine='python')
print(mocodf.head())

              0             1             2         3             4  \
0  1.332086e-14 -1.111059e-14  7.105427e-15  0.000000  2.848359e-35   
1  1.394313e-03  3.979120e-02 -6.179350e-04 -0.000305 -5.038878e-05   
2  4.798411e-03  3.080263e-02 -9.149480e-04 -0.000359 -5.419149e-05   
3  3.124971e-03  5.745299e-02  2.872896e-03 -0.000042  1.485895e-05   
4  5.558033e-04  5.724530e-02 -1.848545e-02  0.000036  8.939434e-05   

              5  
0  2.768145e-18  
1  3.284350e-05  
2  1.160633e-04  
3 -5.470219e-06  
4 -1.247659e-04  


In [ ]:
import matplotlib.pyplot as plt
plt.plot([mocodf[0]])
plt.ylabel('x deviation')
plt.show()

In [ ]:
mocodf[1]

## derivative computation

In [3]:
TR = 2000 
acqNb = 240

def vectorDerivative(v):
    dv = {}
    for i in range(acqNb):
        # print mocodf['x'][i]
        if i== 0:
            dv[i]= (v[i+1]-v[i]) / 2*TR
        elif i== acqNb-1:
            dv[i]= (v[i]-v[i-1]) / 2*TR 
        else:
            dv[i]= (v[i+1]-v[i-1]) / 2*TR
        #print 'derivative' + str(i)
        #print  v[i]
    return v

In [4]:
def plusDerivative(df):
    lg = len(df.columns.values)
    dg = df
    for j in list(df.columns.values):
        vprime = vectorDerivative(df[j])
        dg[lg+j]=vprime
    return dg
    

In [5]:
derivate = plusDerivative(mocodf)
print(derivate.head())

             0             1             2         3             4   \
0  1.332086e-14 -1.111059e-14  7.105427e-15  0.000000  2.848359e-35   
1  1.394313e-03  3.979120e-02 -6.179350e-04 -0.000305 -5.038878e-05   
2  4.798411e-03  3.080263e-02 -9.149480e-04 -0.000359 -5.419149e-05   
3  3.124971e-03  5.745299e-02  2.872896e-03 -0.000042  1.485895e-05   
4  5.558033e-04  5.724530e-02 -1.848545e-02  0.000036  8.939434e-05   

             5             6             7             8         9   \
0  2.768145e-18  1.332086e-14 -1.111059e-14  7.105427e-15  0.000000   
1  3.284350e-05  1.394313e-03  3.979120e-02 -6.179350e-04 -0.000305   
2  1.160633e-04  4.798411e-03  3.080263e-02 -9.149480e-04 -0.000359   
3 -5.470219e-06  3.124971e-03  5.745299e-02  2.872896e-03 -0.000042   
4 -1.247659e-04  5.558033e-04  5.724530e-02 -1.848545e-02  0.000036   

             10            11  
0  2.848359e-35  2.768145e-18  
1 -5.038878e-05  3.284350e-05  
2 -5.419149e-05  1.160633e-04  
3  1.485895e-05 -5

## square

In [ ]:
mocodf[0]**2

In [6]:
def plusSquare(df):
    lg = len(df.columns.values)
    ds = df
    for j in list(df.columns.values):
        vs = df[j]**2
        ds[lg+j]=vs
    return ds    

## 3 - get wm mask (normalised) and calculate signal mean

In [33]:
# input from segment + normalyse
wmMask = '/homes_unix/hirsch/_pypipe/datadir/data_results/structural/norm_files/wc2t0009_t1_s03.nii'

In [34]:
from nipype.interfaces.fsl.maths import MathsCommand

In [35]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

# Start of the new subworkflow

In [43]:
# creation of a subworflow to calculate the bandpass parameters
prebandpass = pe.Workflow(name='prebandpass')

In [44]:
# input node get the good files first the tissues normalised files
sourcedir = '/homes_unix/hirsch/_pypipe/datadir/data_results/structural/norm_files'
# second the merged functionnal file
sourcemergeddir = '/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal'


from nipype import SelectFiles, Node
templates = dict(wmMask=sourcedir+ "/" + "wc2*.nii",
                 lcfMask=sourcedir+ "/" + "wc3*.nii",
                 mergedFile=sourcemergeddir+ "/" + "*_merged.nii.gz")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'lcfMask': <undefined>, 'mergedFile': <undefined>, 'wmMask': <undefined>}

In [45]:
# calculate eroded binary mask for wm
from nipype.interfaces.fsl.maths import MathsCommand

erosion = pe.Node(interface=MathsCommand(), name='erosion')
    
erosion.inputs.args = '-thr 0 -uthr 111 -bin -ero  '     
erosion.inputs.ignore_exception = False     
erosion.inputs.output_type = 'NIFTI_GZ'     
erosion.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"wmMask" , erosion, "in_file")

In [46]:
# calculate eroded binary mask for lcf

erosionLcf = pe.Node(interface=MathsCommand(), name='erosionLcf')
    
erosionLcf.inputs.args = '-thr 0 -uthr 111 -bin -ero '     
erosionLcf.inputs.ignore_exception = False     
erosionLcf.inputs.output_type = 'NIFTI_GZ'     
erosionLcf.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"lcfMask" , erosionLcf, "in_file")

In [47]:
# lets calculate the mean signal on these eroded masks first on wm
from nipype.interfaces.fsl.utils import ImageMeants

wmMeants = Node(ImageMeants(), name="wmMeants")     
wmMeants.inputs.ignore_exception = False     
wmMeants.inputs.order = 1     
wmMeants.inputs.output_type = 'NIFTI_GZ'     
wmMeants.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"mergedFile" , wmMeants, "in_file")   
prebandpass.connect(erosion, "out_file", wmMeants, "mask")


In [48]:
# lets calculate the mean signal on these eroded masks econd on lcf

lcfMeants = Node(ImageMeants(), name="lcfMeants")     
lcfMeants.inputs.ignore_exception = False     
lcfMeants.inputs.order = 1     
lcfMeants.inputs.output_type = 'NIFTI_GZ'     
wmMeants.inputs.terminal_output = 'stream'     
prebandpass.connect(filesource,"mergedFile" , lcfMeants, "in_file")   
prebandpass.connect(erosionLcf, "out_file", lcfMeants, "mask")


In [49]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/homes_unix/hirsch/_pypipe/datadir/data_results'

# for segmented normalised eroded wm and lcf mask
prebandpass.connect(erosion,  'out_file', datasink, 'bandpass')
prebandpass.connect(erosionLcf,  'out_file', datasink, '@1bandpass')

# for wm and lcf mean signal to text files in functionnal repository
prebandpass.connect(wmMeants,  'out_file', datasink, '@5functionnal')
prebandpass.connect(lcfMeants,  'out_file', datasink, '@6functionnal')

In [50]:
# the run
prebandpass.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpi4DxSi/prebandpass/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node erosion in dir: /tmp/tmpRYhaU8/prebandpass/erosion
INFO:workflow:Running: fslmaths /homes_unix/hirsch/_pypipe/datadir/data_results/structural/norm_files/wc2t0009_t1_s03.nii -thr 0 -uthr 111 -bin -ero   /tmp/tmpRYhaU8/prebandpass/erosion/wc2t0009_t1_s03_maths.nii.gz
INFO:workflow:Executing node erosionLcf in dir: /tmp/tmpvP_RBj/prebandpass/erosionLcf
INFO:workflow:Running: fslmaths /homes_unix/hirsch/_pypipe/datadir/data_results/structural/norm_files/wc3t0009_t1_s03.nii -thr 0 -uthr 111 -bin -ero  /tmp/tmpvP_RBj/prebandpass/erosionLcf/wc3t0009_t1_s03_maths.nii.gz
INFO:workflow:Executing node lcfMeants in dir: /tmp/tmpJkBdO6/prebandpass/lcfMeants
INFO:workflow:Running: fslmeants -i /homes_unix/hirsch/_pypipe/datadir/data_results/functionn

# End of precalculations dump in ortho.txt file

In [20]:
import os
def mocoOrtho(f):
    df = pd.read_csv(f, header=None, sep='  ',engine='python')
    dfd = plusDerivative(df)
    dfs = plusSquare(dfd)
    g = dfs.to_csv('ortho.txt', sep=' ', index=False,header=False)
    print g
    h = os.getcwd() + '/' + 'ortho.txt'
    return h

In [21]:
mocoOrtho('/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal/rp_at0009_epi_s04_d0001.txt')

None


'/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal/ortho.txt'

# Brouillons

In [27]:
ImageMeants().help()

Wraps command **fslmeants**

Use fslmeants for printing the average timeseries (intensities) to
the screen (or saves to a file). The average is taken over all voxels in
the mask (or all voxels in the image if no mask is specified)

Inputs::

	[Mandatory]
	in_file: (an existing file name)
		input file for computing the average timeseries
		flag: -i %s, position: 0

	[Optional]
	args: (a string)
		Additional parameters to the command
		flag: %s
	eig: (a boolean)
		calculate Eigenvariate(s) instead of mean (output will have 0 mean)
		flag: --eig
	environ: (a dictionary with keys which are a value of type 'str' and
		 with values which are a value of type 'str', nipype default value:
		 {})
		Environment variables
	ignore_exception: (a boolean, nipype default value: False)
		Print an error message instead of throwing an exception in case the
		interface fails to run
	mask: (an existing file name)
		input 3D mask
		flag: -m %s
	nobin: (a boolean)
		do not binarise the mask for calculation o

In [ ]:
# Node: fsconv.editWmMask     
editWmMask = Node(MathsCommand(), name="editWmMask")     
editWmMask.inputs.args = '-thr 0 -uthr 111 -bin -ero -ero -ero '     
editWmMask.inputs.ignore_exception = False     
editWmMask.inputs.output_type = 'NIFTI_GZ'     
editWmMask.inputs.terminal_output = 'stream'     
fsconv.connect(fsWm2Nii, "out_file", editWmMask, "in_file")

from web
##############################################################################################
##erode a mask or image by zeroing non-zero voxels when zero voxels found in kernel
##############################################################################################
fslmaths 'mask.nii.gz' -kernel box 5x5x5 -ero 'output_image.nii.gz'

In [14]:
%cd '/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal'
%pwd

/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal


u'/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal'

In [15]:
cwd = os.getcwd()
print cwd

/homes_unix/hirsch/_pypipe/datadir/data_results/functionnal


In [ ]:
square = plusSquare(derivate)
print(square.head())

In [ ]:
vectorSquare(mocodf[0])

In [ ]:
list(mocodf.columns.values)

In [ ]:
vectorderivative(mocodf[0])

In [ ]:
TR = 2000 
acqNb = 240

dx = {}
for i in range(acqNb):
    # print mocodf['x'][i]
    if i== 0:
        dx[i]= (mocodf['x'][i+1]-mocodf['x'][i]) / 2*TR
    elif i== acqNb-1:
        dx[i]= (mocodf['x'][i]-mocodf['x'][i-1]) / 2*TR 
    else:
        dx[i]= (mocodf['x'][i+1]-mocodf['x'][i-1]) / 2*TR
    print 'derivative' + str(i)
    print  dx[i]


In [ ]:
sdx = pd.Series(dx)

In [ ]:
sdx